In [1]:
# import libraries
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import xesmf as xe
import cartopy.crs as ccrs
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import matplotlib.colors as colors
from dask.diagnostics import ProgressBar
%matplotlib inline
%config InlineBackend.figure_format = "pdf"

In [2]:
diff_list = []
perc_list = []
models = ['ACCESS1-0','ACCESS1-3','CNRM-CM5','IPSL-CM5B-LR', 'GFDL-ESM2G',
         'MIROC-ESM', 'FGOALS-g2','bcc-csm1-1','BNU-ESM','CanESM2','CCSM4',
          'CSIRO-Mk3-6-0','FGOALS-s2','MIROC5','MPI-ESM-LR','MPI-ESM-P','MRI-CGCM3','NorESM1-M']
for mod in models:
    print(mod)
    # read in last 50 years of piControl
    sic = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/piControl/sic_OImon_'+mod+'_*.nc',
                              parallel=True,combine='by_coords',use_cftime=True).sic.isel(time=
                                                                          slice(-600,None))/100
    areacello = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/piControl/areacello_fx_'+mod+'_*.nc',
                              parallel=True,combine='by_coords',use_cftime=True).areacello
    if(mod in ['CanESM2','CSIRO-Mk3-6-0']):
        areacello = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/piControl/areacella_fx_'+mod+'_*.nc',
                              parallel=True,combine='by_coords',use_cftime=True).areacella
    if(mod in ['FGOALS-g2','FGOALS-s2','BNU-ESM','CSIRO-Mk3-6-0']):
        if(mod=='FGOALS-s2'):
            sic = sic.isel(lon=slice(1,None))
        areacello['lat'] = sic.lat
        areacello['lon'] = sic.lon
    sia = np.sum((sic.where(sic.lat>0) * areacello.where(areacello.lat>0)),axis=[1,2])
    clim = sia.groupby(sia.time.dt.month).mean(dim='time')
    
    sic = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/abrupt4xCO2/sic_OImon_'+mod+'_*.nc',
                              parallel=True,combine='by_coords',use_cftime=True).sic.isel(time=
                                                                          slice(None,1800))/100
    sia = np.sum((sic.where(sic.lat>0) * areacello.where(areacello.lat>0)),axis=[1,2])
    
    with ProgressBar():
        diff = (sia - np.tile(clim,(150))).compute()
    
    diff['time'] = np.arange(0,1800,1)
    perc = (diff / np.tile(clim,(150))).compute() * 100
    diff_list.append(diff)
    perc_list.append(perc)

diff = xr.concat(diff_list,dim='model')
perc = xr.concat(perc_list,dim='model')

ACCESS1-0
[########################################] | 100% Completed |  0.9s
[########################################] | 100% Completed |  2.4s
ACCESS1-3
[########################################] | 100% Completed |  0.9s
[########################################] | 100% Completed |  2.3s
CNRM-CM5
[########################################] | 100% Completed |  0.9s
[########################################] | 100% Completed |  2.5s
IPSL-CM5B-LR
[########################################] | 100% Completed |  0.4s
[########################################] | 100% Completed |  0.7s
GFDL-ESM2G
[########################################] | 100% Completed |  1.0s
[########################################] | 100% Completed |  1.2s
MIROC-ESM
[########################################] | 100% Completed |  0.5s
[########################################] | 100% Completed |  1.3s
FGOALS-g2
[########################################] | 100% Completed |  0.4s
[########################################] 

In [3]:
fig, axes = plt.subplots(ncols=2,figsize=(7.4,3))

# show monthly change in SAT
diff_mon = diff.isel(time=slice(0,12))/1e12
perc_mon = perc.isel(time=slice(0,12))

# plot away!
axes[0].plot(diff_mon.time+1,diff_mon.mean(dim='model'),color='green')
axes[0].fill_between(diff_mon.time+1,diff_mon.mean(dim='model')-diff_mon.std(dim='model'),
                      diff_mon.mean(dim='model')+diff_mon.std(dim='model'),color='green',alpha=0.4)


axes[0].set_xlim([1,12])
axes[0].set_xticks(np.arange(1,13,1))
axes[0].yaxis.set_tick_params(which='both', labelleft=True)
axes[0].tick_params(labelsize=8)
axes[0].set_ylabel('Change in sea ice area (10$^{12}$m$^2$)',fontsize=8)
axes[0].set_xlabel('Months since 4xCO$_2$',fontsize=8)
axes[0].grid()
axes[0].set_title('(a) absolute change',fontsize=8)
axes[1].set_title('(b) percent change',fontsize=8)

# plot away!
axes[1].plot(perc_mon.time+1,perc_mon.mean(dim='model'),color='green')
axes[1].fill_between(perc_mon.time+1,perc_mon.mean(dim='model')-perc_mon.std(dim='model'),
                      perc_mon.mean(dim='model')+perc_mon.std(dim='model'),color='green',alpha=0.4)

axes[1].set_xlim([1,12])
axes[1].set_xticks(np.arange(1,13,1))
axes[1].yaxis.set_tick_params(which='both', labelleft=True)
axes[1].tick_params(labelsize=8)
axes[1].set_ylabel('% change in sea ice area',fontsize=8)
axes[1].set_xlabel('Months since 4xCO$_2$',fontsize=8)
axes[1].grid()

plt.tight_layout()
plt.show()
# plt.savefig('/home/tylerj/CMIP5_AA_final/Fig_1_060820.pdf')

<Figure size 532.8x216 with 2 Axes>

In [4]:
t_3mon = diff.isel(time=slice(0,3)).mean(dim=['model','time']) / (diff.isel(time=slice(0,3)).mean(dim='time').std(dim='model') / np.sqrt(len(models)))
t_12mon = diff.isel(time=slice(0,12)).mean(dim=['model','time']) / (diff.isel(time=slice(0,12)).mean(dim='time').std(dim='model') / np.sqrt(len(models)))

In [11]:
# let's make a plot just of the percent change in sea ice area for the first 30 months

fig, ax = plt.subplots(figsize=(3.8,3))

perc_mon = perc.isel(time=slice(0,30))

ax.hlines(0,1,30,color='black',linestyle=':')

ax.plot(perc_mon.time+1,perc_mon.mean(dim='model'),color='green')
ax.fill_between(perc_mon.time+1,perc_mon.mean(dim='model')-perc_mon.std(dim='model'),
                      perc_mon.mean(dim='model')+perc_mon.std(dim='model'),color='green',alpha=0.4)

ax.set_xlim([1,30])
ax.set_xticks(np.arange(3,31,3))
ax.yaxis.set_tick_params(which='both', labelleft=True)
ax.tick_params(labelsize=8)
ax.set_ylabel('% change in sea ice area',fontsize=8)
ax.set_xlabel('Months since 4xCO$_2$',fontsize=8)
ax.grid()


plt.tight_layout()
# plt.show()
plt.savefig('/home/tylerj/CMIP5_AA_final/perc_change_sia_062320.pdf')

<Figure size 273.6x216 with 1 Axes>